### Set up

In [1]:
import wrds

In [2]:
conn = wrds.Connection(wrds_username='fanjia')

Loading library list...
Done


In [9]:
conn.list_tables(library='comp')

['aco_amda',
 'aco_imda',
 'aco_indfnta',
 'aco_indfntq',
 'aco_indfntytd',
 'aco_indsta',
 'aco_indstq',
 'aco_indstytd',
 'aco_notesa',
 'aco_notesq',
 'aco_notessa',
 'aco_notesytd',
 'aco_pnfnda',
 'aco_pnfndq',
 'aco_pnfndytd',
 'aco_pnfnta',
 'aco_pnfntq',
 'aco_pnfntytd',
 'aco_transa',
 'aco_transq',
 'aco_transsa',
 'aco_transytd',
 'adsprate',
 'asec_amda',
 'asec_imda',
 'asec_notesa',
 'asec_notesq',
 'asec_transa',
 'asec_transq',
 'bank_aacctchg',
 'bank_adesind',
 'bank_afnd1',
 'bank_afnd2',
 'bank_afnddc1',
 'bank_afnddc2',
 'bank_afntind',
 'bank_funda',
 'bank_funda_fncd',
 'bank_fundq',
 'bank_fundq_fncd',
 'bank_iacctchg',
 'bank_idesind',
 'bank_ifndq',
 'bank_ifndytd',
 'bank_ifntq',
 'bank_ifntytd',
 'bank_names',
 'bank_namesq',
 'chars',
 'co_aacctchg',
 'co_aaudit',
 'co_adesind',
 'co_adjfact',
 'co_afnd1',
 'co_afnd2',
 'co_afnddc1',
 'co_afnddc2',
 'co_afntind1',
 'co_afntind2',
 'co_ainvval',
 'co_amkt',
 'co_busdescl',
 'co_cotype',
 'co_filedate',
 'co_

***Directed Provided by Compustat***
~~~python
a = conn.raw_sql("""SELECT
                        datadate,
                        ACCDQ,
                        -- Accrued Expenses & Deferred Income
                        ACCLIY
                        -- Accrued Liabilities - Increase/Decrease
                    FROM
                        comp.g_fundq
                    WHERE
                        gvkey = '019349'
                        AND datadate >= '2019-09-30'::date""")
a
~~~

# Liquidity ratio

**Current Ratio**

In [39]:
# current_ratios quarterly -- q / yearly -- a
current_ratios = conn.raw_sql(
    """SELECT q.datadate, q.actq/q.lctq as current_ratio_q,
              a.act/a.lct as current_ratio_a
        FROM comp.g_fundq q
        JOIN comp.g_funda a
        ON a.gvkey = q.gvkey
        WHERE q.gvkey = '019349'
        AND q.datadate >= '2000-01-01'::date
        ORDER BY a.datadate
        """)
current_ratios

,datadate,current_ratio_q,current_ratio_a
0,2000-03-31,53.342581,4.129180
1,2000-09-30,2.237396,4.129180
2,2001-03-31,1.386260,4.129180
3,2001-09-30,1.145742,4.129180
4,2002-03-31,NaN,4.129180
...,...,...,...
3323,2019-12-31,1.382680,1.552615
3324,2020-03-31,1.577812,1.552615
3325,2020-06-30,1.633536,1.552615
3326,2020-09-30,1.552615,1.552615


In [41]:
# current_ratios / yearly -- a
current_ratios = conn.raw_sql(
    """SELECT datadate, act/lct as current_ratio_a
        FROM comp.g_funda
        WHERE gvkey = '019349'
        AND datadate >= '2016-01-01'::date
        ORDER BY datadate
        """)
current_ratios

,datadate,current_ratio_a
0,2016-09-30,1.289239
1,2017-09-30,1.346476
2,2018-09-30,1.348749
3,2019-09-30,1.387339
4,2020-09-30,1.552615


**Quick Ratio**

In [44]:
# quick_ratio / yearly -- a
quick_ratio = conn.raw_sql("""SELECT datadate, (act - invt)/lct as quick_ratio_a
                            FROM comp.g_funda
                            WHERE gvkey = '019349'
                            AND datadate >= '2016-01-01'::date""")
quick_ratio

,datadate,quick_ratio_a
0,2016-09-30,0.866087
1,2017-09-30,0.886920
2,2018-09-30,1.058717
3,2019-09-30,1.095440
4,2020-09-30,1.324129


**Cash Ratio**

In [68]:
# cash_ratio / yearly -- a
cash_ratio = conn.raw_sql("""SELECT datadate, chee/lct as cash_ratio_a
                            FROM comp.g_funda
                            WHERE gvkey = '019349'
                            AND datadate >= '2016-01-01'::date""")
cash_ratio

,datadate,cash_ratio_a
0,2016-09-30,0.247414
1,2017-09-30,0.193322
2,2018-09-30,0.231148
3,2019-09-30,0.244288
4,2020-09-30,0.411947


**Operating cash flow ratio**

In [52]:
# Operating_cf / yearly -- a
operating_cf = conn.raw_sql("""SELECT datadate, oancf/lct as Operating_cf_ratio_a
                                FROM comp.g_funda
                                WHERE gvkey = '019349'
                                AND datadate >= '2016-01-01'::date""")
operating_cf

,datadate,operating_cf_ratio_a
0,2016-09-30,0.177346
1,2017-09-30,0.165368
2,2018-09-30,0.175983
3,2019-09-30,0.166709
4,2020-09-30,0.259761


# Leverage Ratios

**Debt to equity**

In [58]:
# debt_to_equity / yearly -- a
de_ratio = conn.raw_sql("""SELECT datadate, (dlc + dltt)/teq as de_ratio_a
                                FROM comp.g_funda
                                WHERE gvkey = '019349'
                                AND datadate >= '2016-01-01'::date""")
de_ratio

,datadate,de_ratio_a
0,2016-09-30,0.889447
1,2017-09-30,0.723696
2,2018-09-30,0.669698
3,2019-09-30,0.714905
4,2020-09-30,1.119127


**Debt to asset**

In [60]:
# debt_to_asset/ yearly -- a
da_ratio = conn.raw_sql("""SELECT datadate, (dltt+dlc)/at as da_ratio_a

                            FROM comp.g_funda
                            WHERE gvkey = '019349'
                            AND datadate >= '2016-01-01'::date""")
da_ratio

,datadate,da_ratio_y
0,2016-09-30,0.246323
1,2017-09-30,0.240830
2,2018-09-30,0.231631
3,2019-09-30,0.242586
4,2020-09-30,0.359710


**Debt to EBITDA**

In [62]:
# debt_to_ebitda/ yearly -- a
d_ebita = conn.raw_sql("""SELECT datadate, (dltt+dlc)/(ebitda) as d_ebitda_a
                            FROM comp.g_funda
                            WHERE gvkey = '019349'
                            AND datadate >= '2016-01-01'::date""")
d_ebita

,datadate,d_ebitda_a
0,2016-09-30,3.187545
1,2017-09-30,3.113129
2,2018-09-30,3.521230
3,2019-09-30,3.669754
4,2020-09-30,5.929617


**Net Debt to EBITDA**

In [65]:
# net_debt_to_ebitda/ yearly -- a
net_d_ebitda = conn.raw_sql("""SELECT datadate, (dlttchee)/(ebitda) as net_d_ebitda_a
                                FROM comp.g_funda
                                WHERE gvkey = '019349'
                                AND datadate >= '2016-01-01'::date""")
net_d_ebitda

,datadate,net_d_ebitda_a
0,2016-09-30,2.094596
1,2017-09-30,2.302676
2,2018-09-30,2.310243
3,2019-09-30,2.422171
4,2020-09-30,4.059739


**Interest Coverage Ratio**

In [69]:
# int_cov/ yearly -- a
int_cov = conn.raw_sql("""SELECT datadate, ebitda/xint as int_coverage_a
                            FROM comp.g_funda
                            WHERE gvkey = '019349'
                            AND datadate >= '2016-01-01'::date""")
int_cov

,datadate,int_coverage_a
0,2016-09-30,9.823054
1,2017-09-30,9.848716
2,2018-09-30,8.620755
3,2019-09-30,6.424321
4,2020-09-30,10.381215


**Debt to Total Capitalization**

In [12]:
# d_total_cap/ yearly -- a
d_total_cap = conn.raw_sql("""SELECT datadate, (dltt+dlc)/(dltt+dlc+teq) as d_total_cap_a
                                FROM comp.g_funda
                                WHERE gvkey = '019349'
                                AND datadate >= '2016-01-01'::date""")
d_total_cap

,datadate,d_total_cap_a
0,2016-09-30,0.470745
1,2017-09-30,0.419851
2,2018-09-30,0.401089
3,2019-09-30,0.416877
4,2020-09-30,0.528108


# Efficiency Ratios

**Inventory Turnover**

In [74]:
# invt_turnover/ yearly -- a
invt_turnover = conn.raw_sql("""SELECT datadate, cogs/invt as invt_turnover_a
                                    FROM comp.g_funda
                                    WHERE gvkey = '019349'
                                    AND datadate >= '2016-01-01'::date""")
invt_turnover

,datadate,invt_turnover_a
0,2016-09-30,2.932379
1,2017-09-30,2.758650
2,2018-09-30,3.957868
3,2019-09-30,3.895853
4,2020-09-30,4.360616


Fixed Assets Turnover

In [75]:
# f_asset_turnover/ yearly -- a
f_asset_turnover = conn.raw_sql("""SELECT datadate, revt/ppegt as f_asset_turnover_a
                                    FROM comp.g_funda
                                    WHERE gvkey = '019349'
                                    AND datadate >= '2016-01-01'::date""")
f_asset_turnover

,datadate,f_asset_turnover_a
0,2016-09-30,3.111823
1,2017-09-30,3.089735
2,2018-09-30,2.975528
3,2019-09-30,2.912344
4,2020-09-30,2.447486


**Total Assets Turnover**

In [76]:
# t_asset_turnover/ yearly -- a
t_asset_turnover = conn.raw_sql("""SELECT datadate, revt/at as t_asset_turnover_a
                                    FROM comp.g_funda
                                    WHERE gvkey = '019349'
                                    AND datadate >= '2016-01-01'::date""")
t_asset_turnover

,datadate,t_asset_turnover_a
0,2016-09-30,0.633518
1,2017-09-30,0.620677
2,2018-09-30,0.597804
3,2019-09-30,0.578038
4,2020-09-30,0.461181


# Profitability Ratios

**Gross Profit Margin**

In [78]:
# gross_profit/ yearly -- a
gross_profit = conn.raw_sql("""SELECT datadate, (revt-cogs)/revt as gross_profit_a
                                    FROM comp.g_funda
                                    WHERE gvkey = '019349'
                                    AND datadate >= '2016-01-01'::date""")
gross_profit

,datadate,gross_profit_a
0,2016-09-30,0.331375
1,2017-09-30,0.337584
2,2018-09-30,0.338242
3,2019-09-30,0.335836
4,2020-09-30,0.405117


**Operating Margin**

In [79]:
# operating_margin/ yearly -- a
operating_margin = conn.raw_sql("""SELECT datadate, ebit/revt as operating_margin_a
                                    FROM comp.g_funda
                                    WHERE gvkey = '019349'
                                    AND datadate >= '2016-01-01'::date""")
operating_margin

,datadate,operating_margin_a
0,2016-09-30,0.089661
1,2017-09-30,0.088418
2,2018-09-30,0.071191
3,2019-09-30,0.077053
4,2020-09-30,0.079700


ROA

In [81]:
# roa/ yearly -- a
roa = conn.raw_sql("""SELECT datadate, nicon/at as roa_a
                        FROM comp.g_funda
                        WHERE gvkey = '019349'
                        AND datadate >= '2016-01-01'::date""")
roa

,datadate,roa_a
0,2016-09-30,0.041856
1,2017-09-30,0.044789
2,2018-09-30,0.040910
3,2019-09-30,0.034416
4,2020-09-30,0.033253


**ROE**

In [82]:
# roa/ yearly -- a
roe = conn.raw_sql("""SELECT datadate, nicon/teq as roe_a
                        FROM comp.g_funda
                        WHERE gvkey = '019349'
                        AND datadate >= '2016-01-01'::date""")
roe

,datadate,roe_a
0,2016-09-30,0.151137
1,2017-09-30,0.134592
2,2018-09-30,0.118280
3,2019-09-30,0.101426
4,2020-09-30,0.103458


**Net Profit Margin**

In [83]:
# net_profit / yearly -- a
net_profit = conn.raw_sql("""SELECT datadate, nicon/revt as net_profit_a
                        FROM comp.g_funda
                        WHERE gvkey = '019349'
                        AND datadate >= '2016-01-01'::date""")
net_profit

,datadate,net_profit_a
0,2016-09-30,0.066069
1,2017-09-30,0.072162
2,2018-09-30,0.068434
3,2019-09-30,0.059540
4,2020-09-30,0.072105


**EBITDA Margin**

In [84]:
# ebitda / yearly -- a
ebitda = conn.raw_sql("""SELECT datadate, ebitda/revt as ebitda_a
                        FROM comp.g_funda
                        WHERE gvkey = '019349'
                        AND datadate >= '2016-01-01'::date""")
ebitda

,datadate,ebitda_a
0,2016-09-30,0.121980
1,2017-09-30,0.124637
2,2018-09-30,0.110038
3,2019-09-30,0.114359
4,2020-09-30,0.131539


# Market Value Ratios

**Book value per share**

In [86]:
# bvps / yearly -- a
bvps = conn.raw_sql("""SELECT datadate, ceq/cshpria as bvps_a
                        FROM comp.g_funda
                        WHERE gvkey = '019349'
                        AND datadate >= '2016-01-01'::date""")
bvps

,datadate,bvps_a
0,2016-09-30,42.304430
1,2017-09-30,53.053510
2,2018-09-30,55.792006
3,2019-09-30,59.614282
4,2020-09-30,45.130126


**EPS**

In [14]:
# eps / yearly -- a
eps = conn.raw_sql("""SELECT datadate, nicon/cshpria as eps_a
                        FROM comp.g_funda
                        WHERE gvkey = '019349'
                        AND datadate >= '2016-01-01'::date""")
eps

,datadate,eps_a
0,2016-09-30,6.506852
1,2017-09-30,7.378906
2,2018-09-30,6.972467
3,2019-09-30,6.405516
4,2020-09-30,5.109539


**P/E**

In [32]:
# pe / yearly -- a
pe = conn.raw_sql("""SELECT a.datadate, a.nicon, d.prccd
                        FROM comp.g_funda a
                        JOIN comp.g_sec_dprc d
                        ON a.gvkey = d.gvkey 
                        WHERE a.gvkey = '019349'
                        AND a.datadate >= '2016-01-01'
                        """)
pe

,datadate,nicon,prccd
0,2016-09-30,5262.0,720.00
1,2017-09-30,5993.0,720.00
2,2018-09-30,5683.0,720.00
3,2019-09-30,5171.0,720.00
4,2020-09-30,4120.0,720.00
...,...,...,...
79860,2016-09-30,5262.0,128.33
79861,2017-09-30,5993.0,128.33
79862,2018-09-30,5683.0,128.33
79863,2019-09-30,5171.0,128.33


In [26]:
################### g_sec_dprc 
pe = conn.raw_sql("""SELECT DATE_PART('year', datadate) as date, gvkey, prccd, curcdd
                        FROM comp.g_sec_dprc 
                        WHERE gvkey = '019349'
                        AND DATE_PART('year', datadate) >= '2018'
                        """)
pe.head(50)

,date,gvkey,prccd,curcdd
0,2018.0,019349,121.48,CHF
1,2018.0,019349,121.48,CHF
2,2018.0,019349,121.48,CHF
3,2018.0,019349,121.48,CHF
4,2018.0,019349,121.48,CHF
5,2018.0,019349,121.48,CHF
6,2018.0,019349,121.48,CHF
7,2018.0,019349,121.48,CHF
8,2018.0,019349,121.48,CHF
9,2018.0,019349,121.48,CHF


In [10]:
############################ g_funda
pe = conn.raw_sql("""SELECT datadate, gvkey, nicon
                        FROM comp.g_funda 
                        WHERE gvkey = '019349'
                        AND datadate >= '2018-01-01'
                        """)
pe

,datadate,gvkey,nicon
0,2018-09-30,019349,5683.0
1,2019-09-30,019349,5171.0
2,2020-09-30,019349,4120.0


**P/Cash Flow**

**PEG (Price/EPS)/EPS Growth Rate**

# Others

**Z-Score**

**Credit Ratings**

**Investment Rate (IR)**

**Projected NOPLAT Growth**

**R&D expense / Sales**